In [3]:
import torch
from pde_class.pde_class import Pde
from simple_pinn.simple_pinn import PINN
from helper_functions.helper_functions import plot_loss, get_data

ModuleNotFoundError: No module named 'helper_functions.helper_functions'; 'helper_functions' is not a package

In [ ]:
pde = Pde(E=1e8, nu=0.3, h=0.001, k=5/6)
model = PINN([1, 20, 20, 20, 4], lr=1e-4)

iterations = 100001
report_step = 1000
rel_diff = 100.0
for i in range(iterations):
      model.optimizer.zero_grad()

      # Generate N points
      x = getData(num_points=10, grad=True)
      resloss, derivloss = pde.getLoss(model, x)

      loss = resloss + derivloss
      loss.backward()

      model.optimizer.step()

      with torch.no_grad():
          if i % report_step ==0:
              print(f"iteration {i} resloss {resloss.item():.2e}")
              print(f"iteration {i} derivloss {derivloss.item():.2e}")
              plot_loss(model=model)
              #plot_loss2(model=model)
              #plot_loss3(model=model)
              print